In [ ]:
#### Importing necessary libraries
import pandas as pd
from sklearn import preprocessing
from IPython.display import display, HTML
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.decomposition import PCA


df=pd.read_csv('Base.csv')
print('dataframe shape:',df.shape)
for i in df.columns:
    print(i, ',', df[i].dtype,',',df[i].isnull().any())
display(HTML(df.head(10).to_html()))
print('unique values in source:',df['source'].unique())
print('unique values in our lable:',df['fraud_bool'].unique())

dataframe shape: (1000000, 32)
fraud_bool , int64 , False
income , float64 , False
name_email_similarity , float64 , False
prev_address_months_count , int64 , False
current_address_months_count , int64 , False
customer_age , int64 , False
days_since_request , float64 , False
intended_balcon_amount , float64 , False
payment_type , object , False
zip_count_4w , int64 , False
velocity_6h , float64 , False
velocity_24h , float64 , False
velocity_4w , float64 , False
bank_branch_count_8w , int64 , False
date_of_birth_distinct_emails_4w , int64 , False
employment_status , object , False
credit_risk_score , int64 , False
email_is_free , int64 , False
housing_status , object , False
phone_home_valid , int64 , False
phone_mobile_valid , int64 , False
bank_months_count , int64 , False
has_other_cards , int64 , False
proposed_credit_limit , int64 , False
foreign_request , int64 , False
source , object , False
session_length_in_minutes , float64 , False
device_os , object , False
keep_alive_sessio

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,employment_status,credit_risk_score,email_is_free,housing_status,phone_home_valid,phone_mobile_valid,bank_months_count,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
0,0,0.3,0.986506,-1,25,40,0.006735,102.453711,AA,1059,13096.035020,7850.955007,6742.080561,5,5,CB,163,1,BC,0,1,9,0,1500,0,INTERNET,16.224843,linux,1,1,0,0
1,0,0.8,0.617426,-1,89,20,0.010095,-0.849551,AD,1658,9223.283431,5745.251481,5941.664859,3,18,CA,154,1,BC,1,1,2,0,1500,0,INTERNET,3.363854,other,1,1,0,0
2,0,0.8,0.996707,9,14,40,0.012316,-1.490386,AB,1095,4471.472149,5471.988958,5992.555113,15,11,CA,89,1,BC,0,1,30,0,200,0,INTERNET,22.730559,windows,0,1,0,0
3,0,0.6,0.475100,11,14,30,0.006991,-1.863101,AB,3483,14431.993620,6755.344479,5970.336831,11,13,CA,90,1,BC,0,1,1,0,200,0,INTERNET,15.215816,linux,1,1,0,0
4,0,0.9,0.842307,-1,29,40,5.742626,47.152498,AA,2339,7601.511579,5124.046930,5940.734212,1,6,CA,91,0,BC,1,1,26,0,200,0,INTERNET,3.743048,other,0,1,0,0
5,0,0.6,0.294840,-1,369,30,0.024232,-1.232556,AD,1204,11556.955510,7506.951276,6482.924037,705,5,CB,134,1,BE,1,1,30,0,200,0,INTERNET,6.987316,linux,1,1,0,0
6,0,0.2,0.773085,22,4,40,0.006919,-0.544676,AB,1998,11723.993610,7864.277144,6338.799156,28,8,CA,72,1,BC,1,1,1,0,200,0,INTERNET,28.199923,x11,1,1,0,0
7,0,0.8,0.153880,-1,103,40,0.045122,-1.101184,AB,1548,4999.555801,4526.861667,6426.790817,6,7,CA,163,0,BE,1,1,25,1,200,0,INTERNET,11.234264,other,1,1,0,0
8,0,0.3,0.523655,21,2,30,0.035206,-0.955737,AB,1781,6979.994002,4335.685346,6624.957942,2,10,CA,35,0,BC,1,0,2,0,200,0,INTERNET,5.329387,other,1,1,0,0
9,0,0.8,0.834475,-1,134,20,0.017245,-1.356393,AD,3113,7549.992086,6273.922110,6312.998835,14,20,CA,201,1,BD,1,1,15,0,1500,0,INTERNET,4.103970,other,1,1,0,0


unique values in source: ['INTERNET' 'TELEAPP']
unique values in our lable: [0 1]


In [2]:

df_norm=df.copy(deep=True)
display(HTML(df_norm.head(10).to_html()))

# Here we are converting categorical data to numerical data
print(df_norm['payment_type'].dtype)
print(df_norm['employment_status'].dtype)
print(df_norm['housing_status'].dtype)
print(df_norm['device_os'].dtype)
print(df_norm['source'].dtype)

df_dummies_payment_type=pd.get_dummies(df_norm['payment_type'],dtype=float)
print(df_dummies_payment_type.head(10))
df_dummies_employment_status=pd.get_dummies(df_norm['employment_status'],dtype=float)
print(df_dummies_employment_status.head(10))
df_dummies_housing_status=pd.get_dummies(df_norm['housing_status'],dtype=float)
print(df_dummies_housing_status.head(10))
df_dummies_device_os=pd.get_dummies(df_norm['device_os'],dtype=float)
print(df_dummies_device_os.head(10))
df_dummies_source=pd.get_dummies(df_norm['source'],dtype=float)
print(df_dummies_source.head(10))

# Here we are addimg binary variables to dataframe
df_norm=df_norm.join(df_dummies_payment_type)
df_norm=df_norm.join(df_dummies_employment_status)
df_norm=df_norm.join(df_dummies_housing_status)
df_norm=df_norm.join(df_dummies_device_os)
df_norm=df_norm.join(df_dummies_source)

# Here we are removing the original categorical variable
df_norm=df_norm.drop('payment_type', axis=1)
df_norm=df_norm.drop('employment_status', axis=1)
df_norm=df_norm.drop('housing_status', axis=1)
df_norm=df_norm.drop('device_os', axis=1)
df_norm=df_norm.drop('source', axis=1)

display(HTML(df_norm.head(10).to_html()))

# Here we are doing N-1 binary variable so we are dropping 1 column
df_norm=df_norm.drop('AB', axis=1)
df_norm=df_norm.drop('CA', axis=1)
df_norm=df_norm.drop('BC', axis=1)
df_norm=df_norm.drop('other', axis=1)
df_norm=df_norm.drop('TELEAPP', axis=1)


display(HTML(df_norm.head(10).to_html()))

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,employment_status,credit_risk_score,email_is_free,housing_status,phone_home_valid,phone_mobile_valid,bank_months_count,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
0,0,0.3,0.986506,-1,25,40,0.006735,102.453711,AA,1059,13096.035020,7850.955007,6742.080561,5,5,CB,163,1,BC,0,1,9,0,1500,0,INTERNET,16.224843,linux,1,1,0,0
1,0,0.8,0.617426,-1,89,20,0.010095,-0.849551,AD,1658,9223.283431,5745.251481,5941.664859,3,18,CA,154,1,BC,1,1,2,0,1500,0,INTERNET,3.363854,other,1,1,0,0
2,0,0.8,0.996707,9,14,40,0.012316,-1.490386,AB,1095,4471.472149,5471.988958,5992.555113,15,11,CA,89,1,BC,0,1,30,0,200,0,INTERNET,22.730559,windows,0,1,0,0
3,0,0.6,0.475100,11,14,30,0.006991,-1.863101,AB,3483,14431.993620,6755.344479,5970.336831,11,13,CA,90,1,BC,0,1,1,0,200,0,INTERNET,15.215816,linux,1,1,0,0
4,0,0.9,0.842307,-1,29,40,5.742626,47.152498,AA,2339,7601.511579,5124.046930,5940.734212,1,6,CA,91,0,BC,1,1,26,0,200,0,INTERNET,3.743048,other,0,1,0,0
5,0,0.6,0.294840,-1,369,30,0.024232,-1.232556,AD,1204,11556.955510,7506.951276,6482.924037,705,5,CB,134,1,BE,1,1,30,0,200,0,INTERNET,6.987316,linux,1,1,0,0
6,0,0.2,0.773085,22,4,40,0.006919,-0.544676,AB,1998,11723.993610,7864.277144,6338.799156,28,8,CA,72,1,BC,1,1,1,0,200,0,INTERNET,28.199923,x11,1,1,0,0
7,0,0.8,0.153880,-1,103,40,0.045122,-1.101184,AB,1548,4999.555801,4526.861667,6426.790817,6,7,CA,163,0,BE,1,1,25,1,200,0,INTERNET,11.234264,other,1,1,0,0
8,0,0.3,0.523655,21,2,30,0.035206,-0.955737,AB,1781,6979.994002,4335.685346,6624.957942,2,10,CA,35,0,BC,1,0,2,0,200,0,INTERNET,5.329387,other,1,1,0,0
9,0,0.8,0.834475,-1,134,20,0.017245,-1.356393,AD,3113,7549.992086,6273.922110,6312.998835,14,20,CA,201,1,BD,1,1,15,0,1500,0,INTERNET,4.103970,other,1,1,0,0


object
object
object
object
object
    AA   AB   AC   AD   AE
0  1.0  0.0  0.0  0.0  0.0
1  0.0  0.0  0.0  1.0  0.0
2  0.0  1.0  0.0  0.0  0.0
3  0.0  1.0  0.0  0.0  0.0
4  1.0  0.0  0.0  0.0  0.0
5  0.0  0.0  0.0  1.0  0.0
6  0.0  1.0  0.0  0.0  0.0
7  0.0  1.0  0.0  0.0  0.0
8  0.0  1.0  0.0  0.0  0.0
9  0.0  0.0  0.0  1.0  0.0
    CA   CB   CC   CD   CE   CF   CG
0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
1  1.0  0.0  0.0  0.0  0.0  0.0  0.0
2  1.0  0.0  0.0  0.0  0.0  0.0  0.0
3  1.0  0.0  0.0  0.0  0.0  0.0  0.0
4  1.0  0.0  0.0  0.0  0.0  0.0  0.0
5  0.0  1.0  0.0  0.0  0.0  0.0  0.0
6  1.0  0.0  0.0  0.0  0.0  0.0  0.0
7  1.0  0.0  0.0  0.0  0.0  0.0  0.0
8  1.0  0.0  0.0  0.0  0.0  0.0  0.0
9  1.0  0.0  0.0  0.0  0.0  0.0  0.0
    BA   BB   BC   BD   BE   BF   BG
0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
1  0.0  0.0  1.0  0.0  0.0  0.0  0.0
2  0.0  0.0  1.0  0.0  0.0  0.0  0.0
3  0.0  0.0  1.0  0.0  0.0  0.0  0.0
4  0.0  0.0  1.0  0.0  0.0  0.0  0.0
5  0.0  0.0  0.0  0.0  1.0  0.0  0.0
6 

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,credit_risk_score,email_is_free,phone_home_valid,phone_mobile_valid,bank_months_count,has_other_cards,proposed_credit_limit,foreign_request,session_length_in_minutes,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month,AA,AB,AC,AD,AE,CA,CB,CC,CD,CE,CF,CG,BA,BB,BC,BD,BE,BF,BG,linux,macintosh,other,windows,x11,INTERNET,TELEAPP
0,0,0.3,0.986506,-1,25,40,0.006735,102.453711,1059,13096.035020,7850.955007,6742.080561,5,5,163,1,0,1,9,0,1500,0,16.224843,1,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0,0.8,0.617426,-1,89,20,0.010095,-0.849551,1658,9223.283431,5745.251481,5941.664859,3,18,154,1,1,1,2,0,1500,0,3.363854,1,1,0,0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0,0.8,0.996707,9,14,40,0.012316,-1.490386,1095,4471.472149,5471.988958,5992.555113,15,11,89,1,0,1,30,0,200,0,22.730559,0,1,0,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0,0.6,0.475100,11,14,30,0.006991,-1.863101,3483,14431.993620,6755.344479,5970.336831,11,13,90,1,0,1,1,0,200,0,15.215816,1,1,0,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,0.9,0.842307,-1,29,40,5.742626,47.152498,2339,7601.511579,5124.046930,5940.734212,1,6,91,0,1,1,26,0,200,0,3.743048,0,1,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
5,0,0.6,0.294840,-1,369,30,0.024232,-1.232556,1204,11556.955510,7506.951276,6482.924037,705,5,134,1,1,1,30,0,200,0,6.987316,1,1,0,0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
6,0,0.2,0.773085,22,4,40,0.006919,-0.544676,1998,11723.993610,7864.277144,6338.799156,28,8,72,1,1,1,1,0,200,0,28.199923,1,1,0,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
7,0,0.8,0.153880,-1,103,40,0.045122,-1.101184,1548,4999.555801,4526.861667,6426.790817,6,7,163,0,1,1,25,1,200,0,11.234264,1,1,0,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
8,0,0.3,0.523655,21,2,30,0.035206,-0.955737,1781,6979.994002,4335.685346,6624.957942,2,10,35,0,1,0,2,0,200,0,5.329387,1,1,0,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
9,0,0.8,0.834475,-1,134,20,0.017245,-1.356393,3113,7549.992086,6273.922110,6312.998835,14,20,201,1,1,1,15,0,1500,0,4.103970,1,1,0,0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,credit_risk_score,email_is_free,phone_home_valid,phone_mobile_valid,bank_months_count,has_other_cards,proposed_credit_limit,foreign_request,session_length_in_minutes,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month,AA,AC,AD,AE,CB,CC,CD,CE,CF,CG,BA,BB,BD,BE,BF,BG,linux,macintosh,windows,x11,INTERNET
0,0,0.3,0.986506,-1,25,40,0.006735,102.453711,1059,13096.035020,7850.955007,6742.080561,5,5,163,1,0,1,9,0,1500,0,16.224843,1,1,0,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0,0.8,0.617426,-1,89,20,0.010095,-0.849551,1658,9223.283431,5745.251481,5941.664859,3,18,154,1,1,1,2,0,1500,0,3.363854,1,1,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0,0.8,0.996707,9,14,40,0.012316,-1.490386,1095,4471.472149,5471.988958,5992.555113,15,11,89,1,0,1,30,0,200,0,22.730559,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0,0.6,0.475100,11,14,30,0.006991,-1.863101,3483,14431.993620,6755.344479,5970.336831,11,13,90,1,0,1,1,0,200,0,15.215816,1,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0,0.9,0.842307,-1,29,40,5.742626,47.152498,2339,7601.511579,5124.046930,5940.734212,1,6,91,0,1,1,26,0,200,0,3.743048,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,0,0.6,0.294840,-1,369,30,0.024232,-1.232556,1204,11556.955510,7506.951276,6482.924037,705,5,134,1,1,1,30,0,200,0,6.987316,1,1,0,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
6,0,0.2,0.773085,22,4,40,0.006919,-0.544676,1998,11723.993610,7864.277144,6338.799156,28,8,72,1,1,1,1,0,200,0,28.199923,1,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
7,0,0.8,0.153880,-1,103,40,0.045122,-1.101184,1548,4999.555801,4526.861667,6426.790817,6,7,163,0,1,1,25,1,200,0,11.234264,1,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,0,0.3,0.523655,21,2,30,0.035206,-0.955737,1781,6979.994002,4335.685346,6624.957942,2,10,35,0,1,0,2,0,200,0,5.329387,1,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,0,0.8,0.834475,-1,134,20,0.017245,-1.356393,3113,7549.992086,6273.922110,6312.998835,14,20,201,1,1,1,15,0,1500,0,4.103970,1,1,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [3]:
for column in df_norm.columns:
    if column != 'fraud_bool':
        # avoids division by zero for a constant feature
        if df_norm[column].max() - df_norm[column].min() != 0:
            df_norm[column] = 1 + 4 * ((df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min()))
        else:
             # Handle constant features by setting them to a specific value
            df_norm[column]=1
display('df_norm:',HTML(df_norm.head(10).to_html()))

df_norm1=df_norm          


'df_norm:'

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,credit_risk_score,email_is_free,phone_home_valid,phone_mobile_valid,bank_months_count,has_other_cards,proposed_credit_limit,foreign_request,session_length_in_minutes,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month,AA,AC,AD,AE,CB,CC,CD,CE,CF,CG,BA,BB,BD,BE,BF,BG,linux,macintosh,windows,x11,INTERNET
0,0,2.0,4.946028,1.000000,1.242424,2.5,1.000343,4.673020,1.631736,4.142605,4.192872,4.757560,1.008386,1.512821,3.382826,5.0,1.0,5.0,2.212121,1.0,3.743455,1.0,1.792866,5.0,3.666667,1,1.0,5.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,5.0
1,0,4.5,3.469704,1.000000,1.839161,1.5,1.000515,1.457041,1.989401,3.225226,3.166524,3.989594,1.005031,2.846154,3.318426,5.0,5.0,5.0,1.363636,1.0,3.743455,1.0,1.200870,5.0,3.666667,1,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0
2,0,4.5,4.986831,1.104167,1.139860,2.5,1.000628,1.437091,1.653232,2.099616,3.033333,4.038421,1.025157,2.128205,2.853309,5.0,1.0,5.0,4.757576,1.0,1.020942,1.0,2.092326,1.0,3.666667,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,5.0
3,0,3.5,2.900398,1.125000,1.139860,2.0,1.000356,1.425487,3.079116,4.459067,3.658857,4.017104,1.018449,2.333333,2.860465,5.0,1.0,5.0,1.242424,1.0,1.020942,1.0,1.746420,5.0,3.666667,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,5.0
4,0,5.0,4.369229,1.000000,1.279720,2.5,1.292779,2.951414,2.396029,2.841061,2.863741,3.988701,1.001677,1.615385,2.867621,1.0,5.0,5.0,4.272727,1.0,1.020942,1.0,1.218324,1.0,3.666667,1,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0
5,0,3.5,2.179359,1.000000,4.449883,2.0,1.001235,1.445117,1.718316,3.778027,4.025200,4.508910,2.182390,1.512821,3.175313,5.0,5.0,5.0,4.757576,1.0,1.020942,1.0,1.367659,5.0,3.666667,1,1.0,1.0,1.0,5.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,5.0,1.0,1.0,1.0,5.0
6,0,1.5,4.092343,1.239583,1.046620,2.5,1.000353,1.466532,2.192417,3.817595,4.199366,4.370628,1.046960,1.820513,2.731664,5.0,5.0,5.0,1.242424,1.0,1.020942,1.0,2.344083,5.0,3.666667,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,5.0
7,0,4.5,1.615517,1.000000,1.969697,2.5,1.002300,1.449207,1.923720,2.224709,2.572665,4.455053,1.010063,1.717949,3.382826,1.0,5.0,5.0,4.151515,5.0,1.020942,1.0,1.563148,5.0,3.666667,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0
8,0,2.0,3.094617,1.229167,1.027972,2.0,1.001795,1.453735,2.062845,2.693835,2.479483,4.645186,1.003354,2.025641,2.466905,1.0,5.0,1.0,1.363636,1.0,1.020942,1.0,1.291344,5.0,3.666667,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0
9,0,4.5,4.337900,1.000000,2.258741,1.5,1.000879,1.441262,2.858188,2.828857,3.424206,4.345874,1.023480,3.051282,3.654741,5.0,5.0,5.0,2.939394,1.0,3.743455,1.0,1.234938,5.0,3.666667,1,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0


In [4]:
c=0
c1=0

for i in df_norm.fraud_bool:
    if i:
        c+=1
    else:
        c1+=1
print('samples with lable 0 before resampling:',c)
print('samples with lable 1 before resampling:',c1)

samples with lable 0 before resampling: 11029
samples with lable 1 before resampling: 988971


In [6]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd

# Assuming df_norm contains your dataset with the target variable named 'fraud_bool'
# and 10k rows are positive out of 1 million rows

# Separate features and target variable
y = df_norm['fraud_bool']
x = df_norm.drop('fraud_bool', axis=1)

# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Define the pipeline for hybrid resampling
pipeline = Pipeline([
    ('over', SMOTE(sampling_strategy=0.1)),  # Oversample minority class to 10% of majority class
    ('under', RandomUnderSampler(sampling_strategy=1.0))  # Undersample majority class to balance with minority class
])

# Apply the pipeline to resample the training set
x_train, y_train = pipeline.fit_resample(x_train, y_train)

# Concatenate resampled features and target variable into a DataFrame
df_resampled = pd.concat([pd.DataFrame(x_train, columns=x.columns), pd.Series(y_train, name='fraud_bool')], axis=1)
print("Shape of resampled training set:", df_resampled.shape)

# Print the number of samples in each class after resampling
print("Samples with label 0 after resampling:", sum(y_train == 0))
print("Samples with label 1 after resampling:", sum(y_train == 1))




Shape of resampled training set: (158216, 48)
Samples with label 0 after resampling: 79108
Samples with label 1 after resampling: 79108


In [11]:


# By hold-out evaluations 
from sklearn.model_selection import train_test_split




# build and eval models
from sklearn import neighbors
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import warnings
warnings.filterwarnings("ignore")
from sklearn import neighbors
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix

k_values = [3, 17, 57, 87, 97, 127, 147, 247, 347, 447, 547, 647]
metrics = ['manhattan', 'euclidean']

for metric in metrics:
    print('Metric:', metric)
    for k in k_values:
        clf = neighbors.KNeighborsClassifier(n_neighbors=k, metric=metric, n_jobs=-1)
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        auc = roc_auc_score(y_test, y_pred)
        cm = confusion_matrix(y_test, y_pred)
        tn, fp, fn, tp = cm.ravel()

        print(f'K = {k}, Accuracy = {accuracy:.4f}, Precision = {precision:.4f}, Recall = {recall:.4f}, AUC = {auc:.4f}')
        print(f'Confusion Matrix:\n[[TN = {tn}, FP = {fp}], [FN = {fn}, TP = {tp}]]\n')

Metric: manhattan
K = 3, Accuracy = 0.9628, Precision = 0.9312, Recall = 0.9993, AUC = 0.9628
Confusion Matrix:
[[TN = 13760, FP = 1094], [FN = 10, TP = 14806]]

K = 17, Accuracy = 0.9338, Precision = 0.8923, Recall = 0.9866, AUC = 0.9339
Confusion Matrix:
[[TN = 13089, FP = 1765], [FN = 198, TP = 14618]]

K = 57, Accuracy = 0.9101, Precision = 0.8754, Recall = 0.9561, AUC = 0.9102
Confusion Matrix:
[[TN = 12838, FP = 2016], [FN = 651, TP = 14165]]

K = 87, Accuracy = 0.9012, Precision = 0.8691, Recall = 0.9443, AUC = 0.9012
Confusion Matrix:
[[TN = 12747, FP = 2107], [FN = 825, TP = 13991]]

K = 97, Accuracy = 0.8992, Precision = 0.8679, Recall = 0.9414, AUC = 0.8992
Confusion Matrix:
[[TN = 12731, FP = 2123], [FN = 868, TP = 13948]]

K = 127, Accuracy = 0.8942, Precision = 0.8636, Recall = 0.9358, AUC = 0.8942
Confusion Matrix:
[[TN = 12665, FP = 2189], [FN = 951, TP = 13865]]

K = 147, Accuracy = 0.8910, Precision = 0.8614, Recall = 0.9318, AUC = 0.8911
Confusion Matrix:
[[TN = 1263

In [84]:
#  Logistic regression
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, precision_score, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix as cm

# by hold-out evaluation

clf=LogisticRegression(penalty='l1',solver='liblinear', 
                       max_iter=150)
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('By hold-out evaluation logistic regression: acc = ',acc, ',F1 = ',f1, ',auc = ',auc)

By hold-out evaluation logistic regression: acc =  0.8084086634714555 ,F1 =  0.8089457126724207 ,auc =  0.8084203905448559


In [98]:
# SVM
from sklearn.svm import SVC
import warnings
warnings.filterwarnings("ignore")
# by hold-out evaluation

clf=SVC(kernel='rbf', C=0.1, max_iter=100,degree=2,gamma='auto') 
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)

acc=accuracy_score(y_test, y_pred)

auc = roc_auc_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print('By hold-out evaluation svm: acc = ',acc, ',F1 = ',f1, ',auc = ',auc)

By hold-out evaluation svm: acc =  0.4975631458674594 ,F1 =  0.07642838556187502 ,auc =  0.500282556535134


In [86]:
# random forest
from sklearn import metrics 
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, precision_score, accuracy_score


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier


# by hold-out evaluation

tree = DecisionTreeClassifier(criterion='entropy', max_depth=10, 
                           max_leaf_nodes = 5, ccp_alpha = 0.01,
                          min_samples_split=3,
                          min_impurity_decrease=0.02)
bag = BaggingClassifier(tree, n_estimators=150, max_samples=0.5,bootstrap=False, random_state=100)
clf=bag.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
f1=f1_score(y_test, y_pred)
auc=roc_auc_score(y_test, y_pred)
print('By hold-out evaluation random forest: acc = ',acc, ',F1 = ',f1, ',auc = ',auc)

By hold-out evaluation random forest: acc =  0.7263847600558151 ,F1 =  0.7521342468764884 ,auc =  0.7257946937518287


In [97]:
#  XGBoost model 1


from xgboost import XGBClassifier

clf = XGBClassifier(n_estimators=500,      # Number of boosting rounds (trees)
    random_state=12,       # Random seed for reproducibility
    learning_rate=0.9)    # Step size shrinkage used in update to prevent overfitting  
clf.fit(x_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(x_test)

# Evaluate the model's performance
acc = accuracy_score(y_test, y_pred)
F1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)
print('By hold-out evaluation XGBoost: acc = ', acc, ', F1 = ', F1, ', AUC = ', auc)

By hold-out evaluation XGBoost: acc =  0.947825031850998 , F1 =  0.9472565213835964 , AUC =  0.9479221800684878


In [95]:
# XGBoost model2

clf = XGBClassifier(
    n_estimators=700,      # Number of boosting rounds (trees)
    random_state=12,       # Random seed for reproducibility
    learning_rate=0.7,     # Step size shrinkage used in update to prevent overfitting
    max_depth=6,           # Maximum depth of a tree
    min_child_weight=1,    # Minimum sum of instance weight (hessian) needed in a child
    subsample=0.8,         # Subsample ratio of the training instances
    colsample_bytree=0.8,  # Subsample ratio of columns when constructing each tree
    gamma=0,               # Minimum loss reduction required to make a further partition on a leaf node of the tree
    reg_alpha=0,           # L1 regularization term on weights
    reg_lambda=1,          # L2 regularization term on weights
    scale_pos_weight=1     # Control the balance of positive and negative weights
)
clf.fit(x_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(x_test)

# Evaluate the model's performance
acc = accuracy_score(y_test, y_pred)
F1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)
print('By hold-out evaluation XGBoost: acc = ', acc, ', F1 = ', F1, ', AUC = ', auc)

By hold-out evaluation XGBoost: acc =  0.9472587918865902 , F1 =  0.9467885415816534 , AUC =  0.947344416482728


In [87]:
# Gradient Boosting model 1
from sklearn.ensemble import GradientBoostingClassifier



# Create and fit the Gradient Boosting Classifier
clf = GradientBoostingClassifier(n_estimators=500, random_state=12, learning_rate=0.4)
clf.fit(x_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(x_test)

# Evaluate the model's performance
acc = accuracy_score(y_test, y_pred)
F1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)
print('By hold-out evaluation gradient boosting: acc = ', acc, ', F1 = ', F1, ', AUC = ', auc)

By hold-out evaluation gradient boosting: acc =  0.9519909401605695 , F1 =  0.951252566735113 , AUC =  0.952114118060683


In [90]:
# Gradient Boosting model 2
from sklearn.ensemble import GradientBoostingClassifier



# Create and fit the Gradient Boosting Classifier
clf = GradientBoostingClassifier(n_estimators=500, random_state=12, learning_rate=0.9)
clf.fit(x_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(x_test)

# Evaluate the model's performance
acc = accuracy_score(y_test, y_pred)
F1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)
print('By hold-out evaluation gradient boosting: acc = ', acc, ', F1 = ', F1, ', AUC = ', auc)

By hold-out evaluation gradient boosting: acc =  0.9479059232744849 , F1 =  0.9473361409821319 , AUC =  0.948003313993362
